# Tell Me a Story! - SHAPStories Example

This notebook shows an example of how to generate SHAPstories using Ollama models.

In [1]:
import shap
import pandas as pd
import numpy as np

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from stories import SHAPstory, unwrap
from stories.llm_wrappers import OllamaWrapper
from stories.prompts import make_prompt

/usr/local/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## FIFA Example

### Load data

In [2]:
# Load Data and Split
data = pd.read_csv("../data/FIFA_2018_Statistics.csv")
data = data.merge(
    data[["Date", "Team", "Goal Scored"]],
    left_on=["Date", "Opponent"],
    right_on=["Date", "Team"],
    suffixes=["", "_y"]).drop(columns=["Team_y"]).rename(columns={"Goal Scored_y": "Goal against"})

feature_names = [i for i in data.columns if data[i].dtype in [np.int64, np.int64]]
x = data[feature_names]

y = (data["Man of the Match"] == "Yes")

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

### Train and Compare Accuracy of Various Models

In [3]:
logreg_pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("classifier", LogisticRegression(max_iter=5000, solver="lbfgs"))
])

param_grid = {
    "classifier__C": np.logspace(-3, 3, 13)
}

cv = KFold(n_splits=5, shuffle=True, random_state=42)

logreg_search = GridSearchCV(
    estimator=logreg_pipe,
    param_grid=param_grid,
    cv=cv,
    scoring="roc_auc",   # swap for the metric you care about
    n_jobs=-1,
    refit=True
)

logreg_search.fit(x_train, y_train)
best_logreg = logreg_search.best_estimator_

predictions = logreg_search.predict(x_test)
accuracy = accuracy_score(y_test, predictions)
print("Accuracy:", accuracy)

model = unwrap(logreg_search)
preprocessor = logreg_search.best_estimator_[:-1]
x_test_preprocessed = preprocessor.transform(x_test)
x_test_preprocessed = pd.DataFrame(x_test_preprocessed, columns=x_test.columns)

Accuracy: 0.7307692307692307


Manually Created Descriptions

In [4]:
task_description = """predict whether a football team will have the "Man of the Match" winner in a FIFA Worldcup match, based on the team's statistics """

input_description = "the match"

class_descriptions = {
    0: "class for the team that will not have a player who wins the 'Man of the Match'",
    1: "class for the team that will have a player who wins the 'Man of the Match'"
}

feature_desc = [
    'Number of goals scored by the team during the match.',
    'Percentage of ball possession by the team during the match.',
    'Number of attempts or shots taken by the team.',
    'Number of shots that were on target.',
    'Number of shots that went off target.',
    'Number of shots that were blocked by the opponent.',
    'Number of corner kicks taken by the team.',
    'Number of times the team was caught offside.',
    'Number of free kicks taken by the team.',
    "Number of saves made by the team's goalkeeper.",
    'Percentage of passes that successfully reached a teammate.',
    'Total number of passes made by the team.',
    "Total distance covered by the team's players during the match, in kilometers.",
    'Number of fouls committed by the team.',
    'Number of yellow cards received by the team.',
    'Number of yellow-red cards received by the team.',
    'Number of red cards received by the team.',
    'Number of goals scored by the team during the penalty shootout.',
    "Number of goals that were conceded by the team's goalkeeper.",
]

features_df = pd.DataFrame({
    "Feature name": list(x.columns),
    "Feature description": feature_desc
})

prompt_template = make_prompt(
    task_description,
    input_description,
    class_descriptions,
)

In [5]:
#llm = OllamaWrapper("gemma3:4b")
llm = OllamaWrapper("llama3.1:8b", temperature=0.01)

In [6]:
masker = shap.maskers.Independent(x_test_preprocessed)
explainer = shap.LinearExplainer(model, masker=masker)

Generate Stories for both pre-trained random forest and SVM

In [ ]:
story_generator = SHAPstory(
    logreg_search.best_estimator_, 
    explainer, 
    llm, 
    prompt_template,
    features_df, 
    max_story_features=8,
    only_positive_shaps=True,
    include_linear_model_coefficients=False)
shap_df, predictions_df = story_generator.gen_variables(x_test)

x_test_0 = x_test.iloc[0]
prediction, score = predictions_df.iloc[0].values
shap_values = shap_df.iloc[0].values
prompt = story_generator.generate_prompt(x_test_0, prediction, score, shap_values)

print(prompt)

1.0
0.8556503292834292
An AI model was used to predict whether a football team will have the "Man of the Match" winner in a FIFA Worldcup match, based on the team's statistics .
The input features to the model include data about the match.
The target variable represents one of the following classes:
- class label 0 represents the class for the team that will not have a player who wins the 'Man of the Match'
- class label 1 represents the class for the team that will have a player who wins the 'Man of the Match'

The AI model predicted a certain instance of the dataset to belong to the class 
with label 1 with probability 0.86.

The provided SHAP table was generated to explain this
outcome. It includes every feature along with its value for that instance, and the
SHAP value assigned to it.

The goal of SHAP is to explain the prediction of an instance by
computing the contribution of each feature to the prediction. The
SHAP explanation method computes Shapley values from coalitional game

In [ ]:
stories = story_generator.generate_stories(x_test.iloc[:1])
print(stories[0])

1.0
0.8556503292834292
